In [3]:
import numpy as np
from skimage.measure import label, regionprops
from scipy.sparse import csr_matrix,lil_matrix,coo_matrix
from scipy.linalg import eigh, inv, logm, norm
from  scipy import ndimage,sparse
import cv2
import os
import sys
import csv
import glob

import h5py
from matplotlib import pyplot as plt
import warnings
#warnings.filterwarnings('ignore')


In [32]:
def covd(mat):
    ims = coo_matrix(mat)
    imd = np.pad( mat.astype(float), (1,1), 'constant')
    [x,y,I] = [ims.row,ims.col,ims.data]  

    Ix = [] #first derivative in x
    Iy = [] #first derivative in y
    Ixx = [] #second der in x
    Iyy = [] #second der in y 
    Id = [] #magnitude of the first der 
    Idd = [] #magnitude of the second der
    
    for ind in range(len(I)):
        Ix.append( imd[x[ind]+1,y[ind]] - imd[x[ind]-1,y[ind]] )
        Ixx.append( imd[x[ind]+1,y[ind]] - 2*imd[x[ind],y[ind]] + imd[x[ind]-1,y[ind]] )
        Iy.append( imd[x[ind],y[ind]+1] - imd[x[ind],y[ind]-1] )
        Iyy.append( imd[x[ind],y[ind]+1] - 2*imd[x[ind],y[ind]] + imd[x[ind],y[ind]-1] )
        Id.append(np.linalg.norm([Ix,Iy]))
        Idd.append(np.linalg.norm([Ixx,Iyy]))
    descriptor = np.array( list(zip(list(x),list(y),list(I),Ix,Iy,Ixx,Iyy,Id,Idd)),dtype='int64' ).T     # descriptors
    C = np.cov(descriptor) #covariance of the descriptor

    iu1 = np.triu_indices(C.shape[1]) # the indices of the upper triangular part
    covd2vec = C[iu1]

    return covd2vec

In [ ]:
'''
Set the input information
'''
h5_file = sys.argv[1]   #this file contains the segmented nuclei
datadir = os.path.dirname(os.path.realpath(h5_file))
dapi_file = sys.argv[2] #this file contains the tif images
npz_file = sys.argv[3] #this is the output file with spatial and morphological descriptors
method = sys.argv[4] #choose between covd rotational invariant or not: covdRI or covd 
report = sys.argv[5] #filename of the output report

In [33]:
'''
Set the input information
'''
h5_file = '/home/garner1/pipelines/WSI-analysis/SG/heatmap_module/test_data/iMS342_20190715_001._r20_c23.h5'   #this file contains the segmented nuclei
datadir = os.path.dirname(os.path.realpath(h5_file))
dapi_file = '/home/garner1/pipelines/WSI-analysis/SG/heatmap_module/test_data/iMS342_20190715_001._r20_c23.tif' #this file contains the tif images
npz_file = '/home/garner1/pipelines/WSI-analysis/SG/heatmap_module/test_data/out.npz' #this is the output file with spatial and morphological descriptors
method = 'covd' #choose between covd rotational invariant or not: covdRI or covd 
report = '/home/garner1/pipelines/WSI-analysis/SG/heatmap_module/test_data/report.txt' #filename of the output report

In [34]:
fov = h5py.File(h5_file, 'r') # load the current fov segmentation
mask = fov['/exported_watershed_masks'][:]
mask_reduced = np.squeeze(mask, axis=2) #to get rid of the third dimension
dapi_fov= cv2.imread(dapi_file,cv2.IMREAD_GRAYSCALE) #the dapi tif file of the current FOV

#Check which position the current FOV occupies within the big scan
row = h5_file.split('_r',1)[1].split('_c')[0]
col = h5_file.split('_r',1)[1].split('_c')[1].split('.')[0]

# label all connected components in the fov, 0 is background
mask_label, numb_of_nuclei = label(mask_reduced,return_num=True) 

centroids = []    #list of centroid coordinates for sc in each fov
descriptors = []  #list of descriptors for sc in each fov
counter=0
print('r:',row,'c:',col,'nuclei:',numb_of_nuclei)

for region in regionprops(mask_label,intensity_image=dapi_fov):
    counter+=1
    if ((np.count_nonzero(region.intensity_image) <= 10) or (np.count_nonzero(region.intensity_image) > 2500)) :        #at least 1 cell
        print('The number of pixels is '+str(np.count_nonzero(region.intensity_image))+' in region='+str(counter))
    else:
#        print('The number of pixels is '+str(np.count_nonzero(region.intensity_image))+' in region='+str(counter))
        centroids.append(region.centroid)
        if method == 'covd':
            descriptors.append(covd(region.intensity_image))
        if method == 'covdRI':
            descriptors.append(covd_ri(region.intensity_image))
#save covd to file
from datetime import datetime
# Returns a datetime object containing the local date and time
dateTimeObj = datetime.now()

if numb_of_nuclei > 0:
    np.savez(str(npz_file)+'_'+str(method)+'.npz',centroids=centroids,descriptors=descriptors)
else:
    print('There are no nuclei in row='+str(row)+' and col='+str(col)+' in file: '+str(h5_file))

with open(str(report), 'a+', newline='') as myfile:
     wr = csv.writer(myfile)
     wr.writerow([dateTimeObj,'row='+str(row),'col='+str(col),'nuclei='+str(numb_of_nuclei),'#descriptors='+str(len(descriptors))])
     

r: 20 c: 23 nuclei: 367


In [35]:
filename= str(npz_file)+'_'+str(method)+'.npz'
data = np.load(filename,allow_pickle=True)
covds = data['descriptors']
print(covds.shape)

(367, 45)


In [36]:
print('Clustering the descriptors')
import umap
import hdbscan
import sklearn.cluster as cluster
from sklearn.cluster import OPTICS

# this is used to identify clusters                                 
embedding = umap.UMAP(min_dist=0.0,n_components=3,random_state=42).fit_transform(covds) 

Clustering the descriptors


In [37]:
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px
import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.5, 's' : 80, 'linewidths':0}

import hdbscan

import os
import glob

from sklearn.neighbors import NearestNeighbors
from numpy import linalg as LA
import numpy as np
import pandas as pd

In [38]:
df_embedding = pd.DataFrame(data=embedding, columns=['x','y','z'])
'''
Visualize the 3D UMAP representation of the morphology
'''
fig = px.scatter_3d(df_embedding, x="x", y="y", z="z")
fig.update_traces(marker=dict(size=1,opacity=0.5),selector=dict(mode='markers'))
fig.write_html('test.html', auto_open=True)